In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Castro,BR,-24.7911,-50.0119,58.51,71,3,1.81,clear sky
1,1,Severo-Kurilsk,RU,50.6789,156.1250,31.19,77,86,7.63,overcast clouds
2,2,Verkhoyansk,RU,67.5447,133.3850,-1.80,98,57,1.66,broken clouds
3,3,Rikitea,PF,-23.1203,-134.9692,78.42,69,1,12.66,clear sky
4,4,Provideniya,RU,64.3833,-173.3000,32.11,93,100,24.61,snow


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Rikitea,PF,-23.1203,-134.9692,78.42,69,1,12.66,clear sky
9,9,Souillac,MU,-20.5167,57.5167,77.38,94,75,1.14,light rain
10,10,Vila Velha,BR,-20.3297,-40.2925,78.75,77,0,14.97,clear sky
19,19,Yarada,IN,17.6500,83.2667,84.52,89,75,5.75,broken clouds
20,20,Butaritari,KI,3.0707,172.7902,81.52,74,12,12.35,few clouds
21,21,Cayenne,GF,4.9333,-52.3333,84.24,1,75,14.97,broken clouds
24,24,Tiarei,PF,-17.5333,-149.3333,84.22,73,10,13.20,clear sky
34,34,Puerto Ayora,EC,-0.7393,-90.3518,78.76,88,72,9.95,light rain
35,35,Hithadhoo,MV,-0.6000,73.0833,82.53,75,39,6.17,scattered clouds
37,37,Deogarh,IN,21.5333,84.7333,75.45,75,2,0.60,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                203
City                   203
Country                202
Lat                    203
Lng                    203
Max Temp               203
Humidity               203
Cloudiness             203
Wind Speed             203
Current Description    203
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.replace("", np.NaN).dropna()
clean_df.count()

City_ID                202
City                   202
Country                202
Lat                    202
Lng                    202
Max Temp               202
Humidity               202
Cloudiness             202
Wind Speed             202
Current Description    202
dtype: int64

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Rikitea,PF,78.42,clear sky,-23.1203,-134.9692,
9,Souillac,MU,77.38,light rain,-20.5167,57.5167,
10,Vila Velha,BR,78.75,clear sky,-20.3297,-40.2925,
19,Yarada,IN,84.52,broken clouds,17.6500,83.2667,
20,Butaritari,KI,81.52,few clouds,3.0707,172.7902,
21,Cayenne,GF,84.24,broken clouds,4.9333,-52.3333,
24,Tiarei,PF,84.22,clear sky,-17.5333,-149.3333,
34,Puerto Ayora,EC,78.76,light rain,-0.7393,-90.3518,
35,Hithadhoo,MV,82.53,scattered clouds,-0.6000,73.0833,
37,Deogarh,IN,75.45,clear sky,21.5333,84.7333,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head(10)


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Rikitea,PF,78.42,clear sky,-23.1203,-134.9692,People ThankYou
9,Souillac,MU,77.38,light rain,-20.5167,57.5167,Shanti Maurice Resort & Spa
10,Vila Velha,BR,78.75,clear sky,-20.3297,-40.2925,Hotel Vitória Palace
19,Yarada,IN,84.52,broken clouds,17.6500,83.2667,Yarada Jungle Beach Resorts
20,Butaritari,KI,81.52,few clouds,3.0707,172.7902,Isles Sunset Lodge
21,Cayenne,GF,84.24,broken clouds,4.9333,-52.3333,Hôtel Le Dronmi
24,Tiarei,PF,84.22,clear sky,-17.5333,-149.3333,Le Rocher de Tahiti
34,Puerto Ayora,EC,78.76,light rain,-0.7393,-90.3518,Finch Bay Galapagos Hotel
35,Hithadhoo,MV,82.53,scattered clouds,-0.6000,73.0833,Scoop Guest House
37,Deogarh,IN,75.45,clear sky,21.5333,84.7333,Amit Henej House


In [15]:
len(hotel_df)

202

In [16]:
clean_hotel_df = hotel_df.replace("", np.NaN).dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Rikitea,PF,78.42,clear sky,-23.1203,-134.9692,People ThankYou
9,Souillac,MU,77.38,light rain,-20.5167,57.5167,Shanti Maurice Resort & Spa
10,Vila Velha,BR,78.75,clear sky,-20.3297,-40.2925,Hotel Vitória Palace
19,Yarada,IN,84.52,broken clouds,17.6500,83.2667,Yarada Jungle Beach Resorts
20,Butaritari,KI,81.52,few clouds,3.0707,172.7902,Isles Sunset Lodge
...,...,...,...,...,...,...,...
651,Chapada Dos Guimaraes,BR,78.46,few clouds,-15.4606,-55.7497,Pousada Morro Dos Ventos
653,Carauari,BR,81.43,heavy intensity rain,-4.8828,-66.8958,Hotel Fellip
655,Port Hedland,AU,77.05,scattered clouds,-20.3167,118.5667,The Esplanade Hotel
657,Forecariah,GN,77.27,scattered clouds,9.4306,-13.0881,"Motel "" La Maison Blanche"""


In [17]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
#max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.9)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))